# Preparação de Dados

## Carregar o dataset

Executar o código na célula abaixo para importar os pacotes necessários para executar este notebook.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy.random as nr
import math

%matplotlib inline

Executar o código na célula abaixo para carregar o dataset e imprimir as primeiras linhas do data frame.

In [ ]:
enem = pd.read_csv("../data/projeto_ENEM.csv")
enem.head(20)

## Tratamento dos valores ausentes

Existem 6 linhas com `NU_IDADE` ausente e 5631 linhas com `TP_ESTADO_CIVIL` ausente, que são poucas em relação às 177 mil linhas do dataset, assim decidimos eliminar essas linhas.
